# ***Tarea 5. VaR Ajustado por Liquidez***
### Luis Eduardo Jiménez del Muro - 12/03/2025
---

### Paqueterías

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:,.4f}'.format)


# *Ejercicio 1*

Escribe una función en Python llamada VaR_APL la cual te calculará las métricas de riesgo de un portafolio dado, independientemente de que este sea de Equities, Derivados, Tasas o Forex.
La función recibirá los siguientes inputs:

+ data : Data Frame con los datos de Bid y Ask del portafolio. Solo recibirá estas dos columnas por activo y hará el calculo del Mid Price y el Spread de manera automática. Hay que automatizar el código para que funcione independientemente del número de activos, es decir, si le meto 10 bid y ask o nada más uno tiene que funcionar.

+ posiciones : Array de numpy que contiene la posición por activo. Hay que tener en cuenta que tiene que estar en el mismo ordende data y tener la misma cantidad de posiciones que de número de activos.

+ conf : Float con el nivel de confianza del VaR.

+ long : Valor booleano que es True si estas largo y False si estas corto.

La función deberá realizar todos los calcúlos vistos en clase y regresar un data frame con todas las métricas de riesgo calculadas, es decir, VaR, VaR ajustado por liquidez promedio y VaR ajustado por liquidez estresado. Tanto en porcentaje como en efectivo.

In [121]:
def VaR_APL(df, positions, nc, long):
    bids = [bid for bid in df.columns if 'Bid' in bid]
    asks = [ask for ask in df.columns if 'Ask' in ask]

    last_mid = []

    for i in range(len(bids)):
        df[f'Mid{i}'] = (df[bids[i]] + df[asks[i]]) / 2
        df[f'Spread{i}'] = (df[asks[i]] - df[bids[i]]) / df[f'Mid{i}']
        df[f'Returns{i}'] = df[f'Mid{i}'].pct_change()
        last_mid.append(df[f'Mid{i}'].iloc[-1])

    weights = [weight for weight in (positions * last_mid) / ((last_mid * positions).sum())]
    df['Portfolio_Return'] = np.dot(df[[f'Returns{i}' for i in range(len(bids))]], weights)

    var = np.abs(np.percentile(df['Portfolio_Return'].dropna(), (100-nc)) if long else np.percentile(df['Portfolio_Return'].dropna(), nc))
    cvar = np.abs(df[df['Portfolio_Return'] < -var]['Portfolio_Return'].mean() if long else df[df['Portfolio_Return'] > var]['Portfolio_Return'].mean())

    cl_promedio = np.array([df[col].mean() for col in df.columns if 'Spread' in col])
    cl_estresado = np.array([np.percentile(df[col], 99) for col in df.columns if 'Spread' in col])

    var_apl_promedio, var_apl_estresado = var + cl_promedio.dot(weights), var + cl_estresado.dot(weights)
    cvar_apl_promedio, cvar_apl_estresado = cvar + cl_promedio.dot(weights), cvar + cl_estresado.dot(weights)

    cash_var, cash_cvar = var * (last_mid * positions).sum(), cvar * (last_mid * positions).sum()
    var_apl_promedio_cash, var_apl_estresado_cash = cash_var + cl_promedio.dot(positions*last_mid), cash_var + cl_estresado.dot(positions*last_mid)
    cvar_apl_promedio_cash, cvar_apl_estresado_cash = cl_promedio.dot(positions*last_mid) + cash_cvar, cash_cvar + cl_estresado.dot(positions*last_mid)

    categorias = ['VaR', 'CVaR', 'VaR APL Promedio', 'VaR APL Estresado', 'CVaR APL Promedio', 'CVaR APL Estresado']
    resultados = pd.DataFrame({
        'Métrica': categorias,
        '%': [var, cvar, var_apl_promedio, var_apl_estresado, cvar_apl_promedio, cvar_apl_estresado],
        '$': [cash_var, cash_cvar, var_apl_promedio_cash, var_apl_estresado_cash, cvar_apl_promedio_cash, cvar_apl_estresado_cash]
    })
    return resultados

# *Ejercicio 2*

## Ejercicio 2a

Asume que tienes un portafolio de FX con las siguientes posiciones:
+ CORTO en el dólar 3.2 MDD (USD/MXN)
+ CORTO en el yen japones 10 MDJ (JPY/MXN)
+ CORTO en el euro 5.3 MDE (EUR/MXN)

Calcular las métricas de riesgo de mercado y liquidez del portafolio a un día con un 99% de confianza.
Los datos se encuentran en la hoja FOREX del excel proporcionado:

In [134]:
df = pd.read_excel('data/data_tarea_5.xlsx', sheet_name='FOREX').sort_values('Date')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

positions = np.array([3200000, 10000000, 5300000])
long = False
nc = 99

VaR_APL(df, positions, nc, long)

,Métrica,%,$
0,VaR,0.0261,"4,656,957.0170"
1,CVaR,0.0364,"6,478,384.8304"
2,VaR APL Promedio,0.0266,"4,734,440.2541"
3,VaR APL Estresado,0.0271,"4,824,739.6518"
4,CVaR APL Promedio,0.0368,"6,555,868.0675"
5,CVaR APL Estresado,0.0373,"6,646,167.4652"


## Ejercicio 2b

Asume que tienes una sola posición larga en 100 contratos futuros de granos de café, los contratos tienen un nocional de 37,500 unidades cada uno y los precios de cada unidad (libras) de café se encuentran en el excel proporcionado en la hoja DERIVATIVES. 

Calcular las métricas de riesgo de mercado y liquidez del portafolio a un día con un 99% de confianza.


In [135]:
df = pd.read_excel('data/data_tarea_5.xlsx', sheet_name='DERIVATIVES').sort_values('Date')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

nocional = 37500
positions = np.array([100*nocional])
nc = 99
long = True

VaR_APL(df, positions, nc, long)

,Métrica,%,$
0,VaR,0.0491,"588,274.1440"
1,CVaR,0.0610,"730,740.9163"
2,VaR APL Promedio,0.0498,"596,455.2298"
3,VaR APL Estresado,0.0511,"612,305.6673"
4,CVaR APL Promedio,0.0616,"738,922.0021"
5,CVaR APL Estresado,0.0630,"754,772.4396"
